In [1]:
import pandas as pd
import cx_Oracle

# Replace these values with your Oracle database connection details
username = 'apps'
password = 'apps'
host = 'omsddb.cswg.com'
port = '1521'
service_name = 'omsd_app'

# Create a connection
dsn = cx_Oracle.makedsn(host, port, service_name=service_name)
oms_connection = cx_Oracle.connect(username, password, dsn)

In [2]:
# Retrieve data and create a DataFrame
markout_select = """
    SELECT
        facility_indicator,
        warehouse_loc_num,
        item_number,
        ord_qty_not_ship / 100 ord_qty_not_ship1,
        invoice_number,
        customer_number
    FROM
        oms_stg.oms_wms_ord_delivery_gs_stg wmstg,
        apps.oms_item_master oim,
        apps.oms_division_orgs_loc odo
    WHERE
        1 = 1
        AND wmstg.ebs_process_status = 'N'
        AND wmstg.record_type = 'MK'
        AND oim.wh_item_code = to_number(item_number)
        AND company_code IN ('030', '090')
        AND oim.organization_code IN ('253', '259')
        AND oim.ebs_organization_id = odo.organization_id
        AND rownum = 1
"""

# Execute the SELECT statement
oms_cur = oms_connection.cursor()
oms_cur.execute(markout_select)

# Fetch the results into a DataFrame
oms_markout_df = pd.DataFrame(oms_cur.fetchall(), columns=[
    'facility_indicator',
    'warehouse_loc_num',
    'item_number',
    'ord_qty_not_ship1',
    'invoice_number',
    'customer_number'
])
# print(oms_markout_df)
# Close the cursor
oms_cur.close()
oms_markout_df

,facility_indicator,warehouse_loc_num,item_number,ord_qty_not_ship1,invoice_number,customer_number
0,25,02,40041997,3,000007108,2700


In [4]:
# Replace these values with your Oracle database connection details
username = 'apps'
password = 'apps'
host = 'csebsd2db.cswg.com' #EBSD2
port = '1521'
service_name = 'csebsd2_int' #EBSD2

# Create a connection
dsn = cx_Oracle.makedsn(host, port, service_name=service_name)
ebs_connection = cx_Oracle.connect(username, password, dsn)

In [8]:
# Retrieve data and create a DataFrame
# Assuming you have already created your DataFrame oms_markout_df

# Define the SQL statement with placeholders
# Retrieve the run_id using a sequence (assuming it's a unique identifier)
run_id_sql = 'SELECT XXCMN.XXCMN_FILE_PROC_CONTROL_S.nextval FROM dual'
ebs_cur = ebs_connection.cursor()
ebs_cur.execute(run_id_sql)
run_id = ebs_cur.fetchone()[0]

# Define the SQL statement for insertion
markout_insert = """
    INSERT INTO XXINV.xxinv_inv_adj_from_wms_stg (
        RECORD_ID,
        RECORD_TYPE,
        DISTRIBUTION_CENTER,
        WAREHOUSE_CODE,
        DATE_TIME_CREATED,
        PRODUCT,
        WAREHOUSE_ITEM_CODE,
        PRODUCT_WHSE_CODE,
        QTY_ADJUSTED,
        REASON_CODE,
        CUSTOMER_VENDOR,
        INVOICE,
        RUN_ID,
        CS_ITEM_CODE,
        WMS_SYSTEM,
        CREATED_BY,
        CREATION_DATE,
        LAST_UPDATED_BY,
        LAST_UPDATE_DATE,
        PROCESS_STATUS
    )
    VALUES (
        XXINV.XXINV_INV_ADJ_FROM_WMS_SEQ.NEXTVAL,
        '3001',
        :p1,
        :p2,
        SYSDATE,
        to_number(:p3),
        to_number(:p3),
        :p2,
        to_number((:p4 * (-1))),
        '25',
        :p5,
        :p6,
        to_number(:p7),
        to_number(:p3),
        'EXE',
        -1,
        SYSDATE,
        -1,
        SYSDATE,
        'R'
    )
"""

# Loop through rows of oms_markout_df and execute the SQL statement for each row
ebs_cur.execute("ALTER SESSION SET NLS_DATE_FORMAT = 'YYYY-MM-DD HH24:MI:SS'")
for index, row in oms_markout_df.iterrows():
    bind_values = {
        'p1': row['facility_indicator'],
        'p2': row['warehouse_loc_num'],
        'p3': row['item_number'],
        'p4': row['ord_qty_not_ship1'],
        'p5': row['customer_number'],
        'p6': row['invoice_number'],
        'p7': run_id
    }

    # Execute the SQL statement with the bind values
    ebs_cur.execute(markout_insert, bind_values)

# Commit the changes to the database
ebs_connection.commit()

# Close the cursor and connection
ebs_cur.close()
ebs_connection.close()